In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta,date

# 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set_palette('icefire_r', 2)
import warnings

%matplotlib inline
#Przetwarzanie

# Baza danych gównych do nauki modelu
train = pd.read_csv("train.csv") 
# Baza danych Titanica do sprawdzenia modelu
test = pd.read_csv("test.csv")

In [2]:
train.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [3]:
test.head()

,id,date,country,store,product
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better


In [4]:
print(pd.isnull(train).sum())

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64


In [5]:
print(pd.isnull(test).sum())

id         0
date       0
country    0
store      0
product    0
dtype: int64


In [6]:
cols= ['country', 'store', 'product' ]
for col in cols:
  df_grouped = train.groupby(col)[col].count()
  df_grouped = df_grouped / train[col].count() * 100
  print(f"Unique value counts and percentages for column {col}:")
  print(df_grouped)

Unique value counts and percentages for column country:
country
Argentina    20.0
Canada       20.0
Estonia      20.0
Japan        20.0
Spain        20.0
Name: country, dtype: float64
Unique value counts and percentages for column store:
store
Kagglazon       33.333333
Kaggle Learn    33.333333
Kaggle Store    33.333333
Name: store, dtype: float64
Unique value counts and percentages for column product:
product
Using LLMs to Improve Your Coding                 20.0
Using LLMs to Train More LLMs                     20.0
Using LLMs to Win Friends and Influence People    20.0
Using LLMs to Win More Kaggle Competitions        20.0
Using LLMs to Write Better                        20.0
Name: product, dtype: float64


In [7]:
cols= ['country', 'store', 'product' ]
for col in cols:
  df_grouped = test.groupby(col)[col].count()
  df_grouped = df_grouped / test[col].count() * 100
  print(f"Unique value counts and percentages for column {col}:")
  print(df_grouped)

Unique value counts and percentages for column country:
country
Argentina    20.0
Canada       20.0
Estonia      20.0
Japan        20.0
Spain        20.0
Name: country, dtype: float64
Unique value counts and percentages for column store:
store
Kagglazon       33.333333
Kaggle Learn    33.333333
Kaggle Store    33.333333
Name: store, dtype: float64
Unique value counts and percentages for column product:
product
Using LLMs to Improve Your Coding                 20.0
Using LLMs to Train More LLMs                     20.0
Using LLMs to Win Friends and Influence People    20.0
Using LLMs to Win More Kaggle Competitions        20.0
Using LLMs to Write Better                        20.0
Name: product, dtype: float64


In [8]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [9]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
Enc_col = ['country', 'store', 'product' ]

# Fit the encoder and create the mapping for each column separately
categories = {}
for col in Enc_col:
    for df in [train, test]:
        df[col + '_Enc'] = enc.fit_transform(df[col].values.reshape(-1, 1))
    categories[col] = enc.categories_[0]  # Store categories for the current column

# Print the mapping
for col in Enc_col:
    print(f"Kolumna: {col}")
    for i, category in enumerate(categories[col]):
        print(f"  Wartość zakodowana: {i} -> Wartość początkowa: {category}")

Kolumna: country
  Wartość zakodowana: 0 -> Wartość początkowa: Argentina
  Wartość zakodowana: 1 -> Wartość początkowa: Canada
  Wartość zakodowana: 2 -> Wartość początkowa: Estonia
  Wartość zakodowana: 3 -> Wartość początkowa: Japan
  Wartość zakodowana: 4 -> Wartość początkowa: Spain
Kolumna: store
  Wartość zakodowana: 0 -> Wartość początkowa: Kagglazon
  Wartość zakodowana: 1 -> Wartość początkowa: Kaggle Learn
  Wartość zakodowana: 2 -> Wartość początkowa: Kaggle Store
Kolumna: product
  Wartość zakodowana: 0 -> Wartość początkowa: Using LLMs to Improve Your Coding
  Wartość zakodowana: 1 -> Wartość początkowa: Using LLMs to Train More LLMs
  Wartość zakodowana: 2 -> Wartość początkowa: Using LLMs to Win Friends and Influence People
  Wartość zakodowana: 3 -> Wartość początkowa: Using LLMs to Win More Kaggle Competitions
  Wartość zakodowana: 4 -> Wartość początkowa: Using LLMs to Write Better


In [10]:
categorical_features = ['id','country', 'store', 'product' ]
for df in [train, test]:
     df.drop(columns=categorical_features, inplace=True)

In [11]:
train.tail()

,date,num_sold,country_Enc,store_Enc,product_Enc
136945,2021-12-31,700,4.0,0.0,0.0
136946,2021-12-31,752,4.0,0.0,1.0
136947,2021-12-31,111,4.0,0.0,2.0
136948,2021-12-31,641,4.0,0.0,3.0
136949,2021-12-31,539,4.0,0.0,4.0


In [12]:
test.head()

,date,country_Enc,store_Enc,product_Enc
0,2022-01-01,0.0,1.0,0.0
1,2022-01-01,0.0,1.0,1.0
2,2022-01-01,0.0,1.0,2.0
3,2022-01-01,0.0,1.0,3.0
4,2022-01-01,0.0,1.0,4.0


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         136950 non-null  datetime64[ns]
 1   num_sold     136950 non-null  int64         
 2   country_Enc  136950 non-null  float64       
 3   store_Enc    136950 non-null  float64       
 4   product_Enc  136950 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 5.2 MB


In [14]:
train[['country_Enc', 'store_Enc', 'product_Enc']] = train[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
test[['country_Enc', 'store_Enc', 'product_Enc']] = test[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
train['num_sold'] = train['num_sold'].astype(np.int16)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         136950 non-null  datetime64[ns]
 1   num_sold     136950 non-null  int16         
 2   country_Enc  136950 non-null  int8          
 3   store_Enc    136950 non-null  int8          
 4   product_Enc  136950 non-null  int8          
dtypes: datetime64[ns](1), int16(1), int8(3)
memory usage: 1.7 MB


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375 entries, 0 to 27374
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         27375 non-null  datetime64[ns]
 1   country_Enc  27375 non-null  int8          
 2   store_Enc    27375 non-null  int8          
 3   product_Enc  27375 non-null  int8          
dtypes: datetime64[ns](1), int8(3)
memory usage: 294.2 KB


In [17]:
print(train['date'].isna().any()) 

False


In [23]:
dataa, targett = train_loader

ValueError: too many values to unpack (expected 2)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam  # Import Adam optimizer


# Check data types
print(train['date'].dtype)
print(test['date'].dtype)

# Handle missing values (optional)
if train['date'].isna().any():
   print("There are missing values in 'date' column of train data")
    # Choose a handling method (e.g., remove rows, impute values)

if test['date'].isna().any():
   print("There are missing values in 'date' column of test data")
    # Choose a handling method for test data as well

train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['day'] = pd.to_datetime(train['date']).dt.day
test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month
test['day'] = pd.to_datetime(test['date']).dt.day

    # Separate features and target (excluding date features)

def prepare_data_no_look_back(df, categorical_cols, is_training=True):
  # Separate features based on training/prediction mode
  if is_training:
    features = df[categorical_cols]
    target = df['num_sold']
  else:
    features = df[categorical_cols]  # No target for prediction

  X_tensor = torch.from_numpy(features.to_numpy(dtype=float)).float()

  # Create tensors only for training
  if is_training:
    y_tensor = torch.from_numpy(target.to_numpy(dtype=float)).float()  # Assuming num_sold is float
    dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
    return dataset
  else:
    # Return only features for prediction
    return X_tensor
    
# Assuming 'train' is your training dataframe
categorical_cols = ['year', 'month', 'day', "country_Enc", "store_Enc", "product_Enc"]

# Prepare training data
train_dataset   = prepare_data_no_look_back(train.copy(), categorical_cols)  # Avoid modifying original dataframe
test_dataset    = prepare_data_no_look_back(test.copy(), categorical_cols, is_training=False)

# Create a DataLoader for batch training
batch_size = 64  # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(train_dataset[0][1])
A, B =train_loader

datetime64[ns]
datetime64[ns]
tensor(63.)


ValueError: too many values to unpack (expected 2)

In [25]:
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(LSTMModel, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    # Pass through LSTM layers
    x, _ = self.lstm(x)
    # Use last hidden state for prediction
    x = x[:, -1, :]
    # Pass through final linear layer
    out = self.fc(x)
    return out

# Hyperparameters (adjust as needed)
input_size = len(categorical_cols)  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1  # Assuming predicting single value (num_sold)

# Create model and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()  # Mean Squared Error loss for regression

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    # Access features and target explicitly
    features = data
    targets = target
    # Clear gradients
    optimizer.zero_grad()
    # Forward pass
    prediction = model(data)
    # Calculate loss
    loss = loss_fn(prediction, target)
    # Backpropagation
    loss.backward()
    # Update weights
    optimizer.step()

    # Print training progress (optional)
    if (i+1) % 100 == 0:
      print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# # Prediction on test data (example)
# def predict(model, data_loader):
#   predictions = []
#   with torch.no_grad():
#     for data in data_loader:
#       prediction = model(data)
#       predictions.extend(prediction.squeeze(0).numpy())  # Remove batch dimension
#   return predictions

test_predictions = predict(model, test_loader)
print(f'Example prediction: {test_predictions[0]}')

#Kurwa mać no już nie wiem o co tu chodzi aaaaaaaaaaaaaaaaaaaaa

IndexError: too many indices for tensor of dimension 2